# Twitter Following

The purpose of this notebook is to allow you to extract information about the people a single Twitter user follows. In this example we extract followers, following, name, and description.

In [1]:
from __future__ import unicode_literals
import time
import tweepy

ckey='your credentials here'
c_secret='your credentials here'
atoken='your credentials here'
atokensecret='your credentials here'
auth = tweepy.OAuthHandler(ckey, c_secret)
auth.set_access_token(atoken, atokensecret)


#This allows access to the free Twitter API through Tweepy. Tweepy now will automatically sleep, and notify
#you when it is sleeping by setting the two rate limit arguments to True.
api = tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

In [ ]:
#This creates an empty list, and creates a Tweepy cursor which pulls Twitter IDs and places them in the empty list.
ids = []
for page in tweepy.Cursor(api.friends_ids, screen_name="Insert_Handle").pages():
    ids.extend(page)
    time.sleep(60)
#print(ids)

In [ ]:
#This creates four empty lists, and performs API calls on each to extract the relevant information.
#Due to the wait limit for ~4k followers this takes about 4 hours to run.
names=[]
followers=[]
following=[]
description=[]
for x in ids:
    names.append(api.get_user(x).screen_name)
    followers.append(api.get_user(x).followers_count)
    following.append(api.get_user(x).friends_count)
    description.append(api.get_user(x).description)

In [3]:
import pandas as pd

In [ ]:
#This creates a dataframe from the lists above, and makes a copy if you want the untouched data.
df = pd.DataFrame(columns=['names','followers','following','description'])
d = {'Name':names,'Followers':followers, 'Following':following, 'Description':description}
df=pd.DataFrame(d)
df1 = df.copy()

In [ ]:
#This computes some kind of totally made up Klout score, which is just the ratio of Followers to Following.
#The dataframe is then sorted by Klout and Follower count. Then Klout is dropped, because it's totally useless.
df1['Klout'] = df1['Followers']/df1['Following']
df1 = df1.sort_values(by=['Klout','Followers'], ascending=False)
df1 =df1.set_index('Name')
df1 = df1.drop(['Klout'],axis=1)

In [ ]:
#This exports the sorted list into an excel file for easy consumption.
writer = pd.ExcelWriter('Handle_Name_Followers.xlsx')
df1.to_excel(writer,'Handle_Name')
writer.save()